# 循环神经网络
本节介绍循环神经网络，下图展示了如何基于循环神经网络实现语言模型。我们的目的是基于当前的输入与过去的输入序列，预测序列的下一个字符。循环神经网络引入一个隐藏变量$H$，用$H_{t}$表示$H$在时间步$t$的值。$H_{t}$的计算基于$X_{t}$和$H_{t-1}$，可以认为$H_{t}$记录了到当前字符为止的序列信息，利用$H_{t}$对序列的下一个字符进行预测。
![Image Name](https://cdn.kesci.com/upload/image/q5jkm0v44i.png?imageView2/0/w/640/h/640)

## 循环神经网络的构造

我们先看循环神经网络的具体构造。假设$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$是时间步$t$的小批量输入，$\boldsymbol{H}_t  \in \mathbb{R}^{n \times h}$是该时间步的隐藏变量，则：


$$
\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}  + \boldsymbol{b}_h).
$$


其中，$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$，$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$，$\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h}$，$\phi$函数是非线性激活函数。由于引入了$\boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}$，$H_{t}$能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。由于$H_{t}$的计算基于$H_{t-1}$，上式的计算是循环的，使用循环计算的网络即循环神经网络（recurrent neural network）。

在时间步$t$，输出层的输出为：


$$
\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q.
$$


其中$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$，$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$。


## 从零开始实现循环神经网络

我们先尝试从零开始实现一个基于字符级循环神经网络的语言模型，这里我们使用周杰伦的歌词作为语料，首先我们读入数据：

In [16]:
import torch
import torch.nn as nn
import time
import math
# import sys
# sys.path.append("/home/kesci/input")
# import d2l_jay9460 as d2l
# (corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

# open the jaychou_lyrics.txt
# returns:
# corpus_indices: list of idx
# char_to_idx: dictionary maps character to index
# idx_to_char: list maps index to character
# voacab_size: size of vocabulary
def load_data_jay_lyrics():
    with open("C://jupyter_notebook//boyu//jaychou_lyrics.txt") as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace("\n", ' ').replace('\r', ' ').replace("'", " ").replace(" '", " ")
    idx_to_char = list(set(corpus_chars))
    char_to_idx = {char:i for i, char in enumerate(idx_to_char)}
    vocab_size = len(idx_to_char)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### one-hot向量

我们需要将字符表示成向量，这里采用one-hot向量。假设词典大小是$N$，每次字符对应一个从$0$到$N-1$的唯一的索引，则该字符的向量是一个长度为$N$的向量，若字符的索引是$i$，则该向量的第$i$个位置为$1$，其他位置为$0$。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [23]:
# 将一维向量x转为one-hot向量
# 传入参数：
# x：shape: 为(shape[0]行数, shape[1]列数)的tensor
# n_class : 语料库的长度
# 输出：    
#          （x.shape[0], n_class）的one-hot tensor
def one_hot(x, n_class, dtype=torch.float32):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)  # shape: (n, n_class)
#     print("x.long.view(-1, 1): ", x.long().view(-1, 1))
    # scatter_函数的说明：
    # 传入参数 (int dim, Tensor index, Tensor src)
    # 第一个参数指定维度，这里dim=1即为对每一行的某些索引的位置修改为src
    # 第二个参数指定需要修改的元素的索引，因为我们第一个参数指定了dim=1，因此在这里需要传入一个列向量
    #            指定相应的索引，如[0,2]就是我们要将[0, 0]位置和[1,2]位置修改为src
    # 第三个参数指定应该改为什么值
    result.scatter_(1, x.long().view(-1, 1), 1)  # result[i, x[i, 0]] = 1
    return result
    
x = torch.tensor([0, 2])
x_one_hot = one_hot(x, vocab_size)
print(x_one_hot)
print(x_one_hot.shape)
print(x_one_hot.sum(axis=1))  # 每一行求和都是1

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])
torch.Size([2, 1027])
tensor([1., 1.])


我们每次采样的小批量的形状是（批量大小, 时间步数）。下面的函数将这样的小批量变换成数个形状为（批量大小, 词典大小）的矩阵，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为词向量大小，即one-hot向量长度（词典大小）。

In [24]:
# 将x的每个列向量进行one-hot变换
# 输入参数：
# X: 要进行变换的矩阵
# n_class: 语料库的大小
# 返回值： 一个列表，每个元素是X的列向量进行one-hot变换的结果
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)
print(inputs)

5 torch.Size([2, 1027])
[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]


### 初始化模型参数


In [25]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
# num_inputs: d
# num_hiddens: h, 隐藏单元的个数是超参数
# num_outputs: q

# 初始化模型参数
def get_params():
    def _one(shape):
        param = torch.zeros(shape, device=device, dtype=torch.float32)
        nn.init.normal_(param, 0, 0.01)
        return torch.nn.Parameter(param)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
    return (W_xh, W_hh, b_h, W_hq, b_q)

### 定义模型

函数`rnn`用循环的方式依次完成循环神经网络每个时间步的计算。


In [50]:
# 计算：根据输入输出outputs和计算隐层H
# 传入参数：
# inputs: 
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
#         print("shape: X: ", X.shape, "W_xh: ", W_xh.shape, "H: ", H.shape, "W_hh: ", W_hh.shape)
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

函数init_rnn_state初始化隐层变量，这里的返回值是一个元组。


In [51]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。

In [28]:
print(X.shape)
print(num_hiddens)
print(vocab_size)
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(inputs), inputs[0].shape)
print(len(outputs), outputs[0].shape)
print(len(state), state[0].shape)
print(len(state_new), state_new[0].shape)

torch.Size([2, 5])
256
1027
5 torch.Size([2, 1027])
5 torch.Size([2, 1027])
1 torch.Size([2, 256])
1 torch.Size([2, 256])


### 裁剪梯度

循环神经网络中较容易出现梯度衰减或梯度爆炸，这会导致网络几乎无法训练。裁剪梯度（clip gradient）是一种应对梯度爆炸的方法。假设我们把所有模型参数的梯度拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度


$$
 \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}
$$


的$L_2$范数不超过$\theta$。


In [52]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

### 定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。


In [53]:
# 循环调用rnn(X, state, params), 并根据RNN计算步骤得到一个句子
# 
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)  # 初始化隐层变量，下面将不断循环得到新的state
    output = [char_to_idx[prefix[0]]]   # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y[0].argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀“分开”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [32]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开涯杨恼碌补者碎莫若值'

### 困惑度

我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下[“softmax回归”](../chapter_deep-learning-basics/softmax-regression.ipynb)一节中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

### 定义模型训练函数

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

1. 使用困惑度评价模型。
2. 在迭代模型参数前裁剪梯度。
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。

In [54]:
# 上一节的内容
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        # The size of tensor a (27) must match the size of tensor b (32) at non-singleton dimension 0
        # 尽管batch_indices是用下标[i: i + batch_size]取得的，但其长度不一定为batch_size.
        # Python 将对超过这种切片方式的区间右端点right自动做min(x, right)
        if len(batch_indices) != batch_size: continue
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)
        
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # 必须将indices的长度clip成batch_size的倍数，否则会报RuntimeError
    # RuntimeError: shape '[batch_size, -1]' is invalid for input of size 
    
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps  # 列数 / 时间步数 = 可以采样的次数
    # 每一次产生一个抽样
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [35]:
# 在线性回归章节中实现的sgd
def SGD(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

In [55]:
# 训练并且输出RNN的预测值
# 

# 无返回结果，但对每个epoch，输出他的perplexity和time，
# 以及当前epoch对给定前缀prefixes输入给出的结果

def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        # 相邻采样的特点是：两个相邻的batch，在训练数据上也是连续的
        #          因此只需要在每个epoch开始的时候初始化。
        #          带来问题：在每个epoch中，随着batch的增大，模型的损失函数
        #                   关于隐藏变量的梯度传播得更远，计算开销也更大，
        #                   因此为了减少计算开销，我们一般在每个batch开始的时候
        #                   把隐层状态，从计算图中分离出来
        # 
        
        # WHY
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        # l_sum 是loss的总和，用于计算perplexity
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            # inputs是num_steps个形状为(batch_size, vocab_size)的矩阵
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            # 拼接之前outpus 是每个元素为  长度为num_steps的列表
            outputs = torch.cat(outputs, dim=0)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成形状为
            # (num_steps * batch_size,)的向量，这样跟输出的行一一对应
            y = torch.flatten(Y.T)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            SGD(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            # 输出epoch，perplexity和训练时间time
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            # 使用predict_rnn输出训练结果
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

### 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为50个字符（不考虑前缀长度）的一段歌词。我们每过50个迭代周期便根据当前训练的模型创作一段歌词。

In [43]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [56]:
import random
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 51.570266, time 0.63 sec
 - 分开 我想要你的 我不 我不 我不要再想 我不 我不 我不要再想 我不 我不 我不要再想 我不 我不 我
 - 不分开  我去 我不起 爱什么 干颗四颗 在暖村  静有了我不起 快使用双 习谁了空 在谁的空 在人的可 
epoch 100, perplexity 7.146718, time 0.67 sec
 - 分开 一颗是演 一颗心空 在一放空 你一场空 你一了空 你一场空 你一场空 你一场空 你一场空 你一场空
 - 不分开扫 我说想这你  没有你烦我有多难熬多难熬  没有你烦 我试多努力 你人 他念 不著再久了你的老空 
epoch 150, perplexity 2.310857, time 0.86 sec
 - 分开 用够翻  什知著一屋守折 看着的国我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦的黑猫笑起来像哭 啦啦的黑猫笑起来像哭
epoch 200, perplexity 1.465425, time 0.66 sec
 - 分开 能够翻译 如果这一切 真的可以 我想要将我的寂寞封闭 然后在这里 我学往开抽离 如果我遇见你是一场
 - 不分开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不
epoch 250, perplexity 1.267421, time 0.65 sec
 - 分开 能够翻译 如果这一切 真的可以 我想要将我的寂寞封闭 然后在这里 我学往够抽离 如果我遇见你是一场
 - 不分开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不


接下来采用相邻采样训练模型并创作歌词。

In [57]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 44.714573, time 0.80 sec
 - 分开 一要用双我的惯 从小就耳 我有 双截棍 快什么 干什么 三颗四颗三步四颗 什么线 我爱 这生了觉 
 - 不分开 爱要在双我习惯 从小就 快什么 三颗四颗三步四颗 什么线 我爱 这生了觉 快使用双截棍 哼哼哈兮 
epoch 100, perplexity 5.446462, time 0.67 sec
 - 分开 这要开 旧沉我的脚头 有话去对医药 说 随我的人睡着 想物下这生活 我爱你的生活 不知不觉 你已经
 - 不分开只 我想无再想经 我想 你你很考 我想好努恼  没有你烦我有多 就要 没有 我不 这不 我不要 爱情
epoch 150, perplexity 1.810133, time 0.71 sec
 - 分开 爱不开 娘子我的我有着有一险在等离 不过再一个打慢单的回忆 没有了过去 我将往事抽离 如果我遇见你
 - 不分开只不的我 在不休当只是雕花牛 像是一场默剧 你的完美主义 太彻底 分手的话像语言暴力 我已无能为力再
epoch 200, perplexity 1.261786, time 0.64 sec
 - 分开 一只都  被来单脚 一只坊名 在你放纵 恨自己真 你在操纵 恨自己真 没有不动 别有梦动 没有梦 
 - 不分开只果的愿样   在我爱来起 爸晶你说不汉 你说得悄的画面的钟 从反方向开始移动 回到当初爱你的时空 
epoch 250, perplexity 1.155937, time 0.70 sec
 - 分开 爱果都好动咒 让我回看睡 不限你期前然 篮伤妙传出手 漂亮的假动作 帅呆了我 全场盯人防守 篮下禁
 - 不分开永果 我有到你爱你 我就你你不舍的课 我想就这样名 我爱你 相爱我 我想就声宣你 对你依依不舍 连隔


## 循环神经网络的简介实现

### 定义模型

我们使用Pytorch中的`nn.RNN`来构造循环神经网络。在本节中，我们主要关注`nn.RNN`的以下几个构造函数参数：

* `input_size` - The number of expected features in the input x
* `hidden_size` – The number of features in the hidden state h
* `nonlinearity` – The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
* `batch_first` – If True, then the input and output tensors are provided as (batch_size, num_steps, input_size). Default: False

这里的`batch_first`决定了输入的形状，我们使用默认的参数`False`，对应的输入形状是 (num_steps, batch_size, input_size)。

`forward`函数的参数为：

* `input` of shape (num_steps, batch_size, input_size): tensor containing the features of the input sequence. 
* `h_0` of shape (num_layers * num_directions, batch_size, hidden_size): tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided. If the RNN is bidirectional, num_directions should be 2, else it should be 1.

`forward`函数的返回值是：

* `output` of shape (num_steps, batch_size, num_directions * hidden_size): tensor containing the output features (h_t) from the last layer of the RNN, for each t.
* `h_n` of shape (num_layers * num_directions, batch_size, hidden_size): tensor containing the hidden state for t = num_steps.

现在我们构造一个`nn.RNN`实例，并用一个简单的例子来看一下输出的形状。

In [58]:
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)
num_steps, batch_size = 35, 2
X = torch.rand(num_steps, batch_size, vocab_size)
state = None
Y, state_new = rnn_layer(X, state)
print(Y.shape, state_new.shape)

torch.Size([35, 2, 256]) torch.Size([1, 2, 256])


我们定义一个完整的基于循环神经网络的语言模型。

In [59]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, inputs, state):
        # inputs.shape: (batch_size, num_steps)
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X)  # X.shape: (num_steps, batch_size, vocab_size)
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])  # hiddens.shape: (num_steps * batch_size, hidden_size)
        output = self.dense(hiddens)
        return output, state

类似的，我们需要实现一个预测函数，与前面的区别在于前向计算和初始化隐藏状态。

In [60]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

使用权重为随机值的模型来预测一次。

In [61]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开童诉城恨城恨城城城城'

接下来实现训练函数，这里只使用了相邻采样。

In [62]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state[0].detach_()
                    state[1].detach_()
                else: 
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.T)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

训练模型。

In [65]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['爱你', '想你']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.009622, time 0.21 sec
 - 爱你 你不懂 我不懂 一直到我们都睡着 我想就这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带
 - 想你 我叫我的伤爸 你实我 别都能 说你眼你睡不会 多年的回忆就来去一的女容 纪录第一次遇见的你 如果我
epoch 100, perplexity 1.009905, time 0.21 sec
 - 爱你 开不了 我著你这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 你已
 - 想你 我轻轻的叹息 后悔着对不起 藤蔓植物 爬满了伯爵的坟墓 古堡里一片荒芜 长满杂草的泥土 不会骑扫把
epoch 150, perplexity 1.008857, time 0.22 sec
 - 爱你 我不中 三颗三颗三颗三颗三颗四颗 连成线乘著风 游荡在蓝天边 一片云掉落在我面前 捏成你的形状 随
 - 想你 我叫我的伤爸 你实我 想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每
epoch 200, perplexity 1.008624, time 0.21 sec
 - 爱你 我不中 我不懂不 我不能再想 用不还我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我
 - 想你 我不懂  能再这样的老限日期 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁
epoch 250, perplexity 1.008916, time 0.22 sec
 - 爱你 我不再 不想活 我没有你烦 我有多烦恼  没有你烦我有多烦恼多难熬  穿过云层 我试著努力向你奔跑
 - 想你 我不再  你叫 一只会我 睡著  想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙
